In [ ]:
from __future__ import print_function
from __future__ import unicode_literals
import yaml
import json
import io
import procesamiento_microdatos as pm
import pandas as pd
import unicodecsv as csv
import openpyxl as pyxl
from collections import OrderedDict

In [ ]:
panel = pm.generate_panel()

In [ ]:
with open("grupos.yaml") as groups_file:
    GRUPOS = yaml.load(groups_file)
    
with open("alias.yaml") as alias_file:
    ALIAS = yaml.load(alias_file)
    
AGRUPACION_USOS_PROPIA = {
    "Otros Conceptos de Oferta": ['Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes'],
    "Centrales Eléctricas": ['Servicio Público', 'Autoproducción'],
    "Otros Centros de Transformación": ['Aceiteras y Destilerías', 'Coquerías', 'Carboneras', 'Altos Hornos']
}

AGRUPACION_OFERTA_INTERNA = ["Producción", "Importación", "Exportación", "Pérdidas", 'Variación de Stock', 'Búnker', 'No Aprovechado', 'Ajustes']

AGRUPAMIENTOS_ENERGIAS_MINEM = {
    "Coque": ["Coque de Carbón", "Coque de Petróleo"],
    "Carbón de Leña": ["Carbón Vegetal"],
    "No Energéticos": ["No Energético", "No Energético de Carbón", "Etano"]
}

NODOS_BASE = get_nodos("dict")

In [ ]:
def unalias(df, alias=ALIAS):
    """Usa el dict 'alias' para dar nombres completos al índice y las columnas de un DataFrame"""
    return df.rename(columns=alias, index=alias)

def calcular_perdidas(data):
    for uso in data.columns.get_values():
        if uso in agrupamientos_usos["Centros de Transformación"]:
            data.loc["Pérdidas por Transformación", uso] = -sum(data[uso].dropna())
        else:
            data.loc["Pérdidas por Transformación", uso] = 0
    return data

def sumar_filas_df(df, nueva_fila, filas, borrar=True):
    """Suma todas las filas con índices en la lista `filas`,
    en una nueva fila con índice `nueva_fila`. Si `borrar`, no las incluye en el df retornado. Devuelve el df"""
    df.loc[nueva_fila] = reduce(pd.Series.add, [df.loc[f] for f in filas])
    if borrar:
        df = df.drop(filas, axis=0)
    return df
        
def adaptar_df_a_entidades_minem(df):
    for energia in AGRUPAMIENTOS_ENERGIAS_MINEM:
        df = sumar_filas_df(df, nueva_fila=energia, filas=AGRUPAMIENTOS_ENERGIAS_MINEM[energia])
    return df

def simplificar_usos(df):
    df_usos = df.transpose()
    df_usos = sumar_filas_df(df_usos, nueva_fila="Oferta Interna", filas=AGRUPACION_OFERTA_INTERNA, borrar=False)
    for uso in AGRUPACION_USOS_PROPIA:
        df_usos = sumar_filas_df(df_usos, nueva_fila=uso, filas=AGRUPACION_USOS_PROPIA[uso])
    df = df_usos.transpose()
    return df

def get_yr(yr):
    """Devuelve la data correspondiente a un año del panel de microdatos con los nombres de energías y usos completos."""
    df = unalias(panel[yr])
    df = corregir_signo_consumo(df)
    df = calcular_perdidas(df)
    df = adaptar_df_a_entidades_minem(df)
    df = simplificar_usos(df)
    return df

def corregir_signo_consumo(df):
    # Corrijo signo de rubros de consumo para que "reciban" de las distintas formas de energía
    for consumo in ["Consumo No Energético", "Residencial", "Comercial", "Transporte", "Agropecuario", "Industria"]:
        df[consumo] = -df[consumo]
    return df

def make_links(yr):
    df = get_yr(yr)
    links = list()
    # Genero links directamente desde los microdatos cuando es posible
    for energia in df.index.get_values() if energia !=:
        for col in data.columns.get_values():
            value = data.loc[row, col]
            if value < 0:
                links.append({"source": row, "target": col, "value": abs(value)})
            elif value > 0:
                links.append({"source": col, "target": row, "value": abs(value)})
                
    return links

def get_nodos(formato="dict"):
    wb = pyxl.load_workbook("maestro-nodos.xlsx")
    ws = wb.active
    raw_data = ws.values
    
    cols = next(raw_data)
    rows = list(raw_data)
    if formato == "dict":
        return [dict(zip(cols, row)) for row in rows]
    elif formato == "df":
        return pd.DataFrame.from_records(rows, columns=cols)
    else:
        print("Formato no reconodico: {}".format(formato))

nodos = get_nodos("df")

In [ ]:
data2015 = get_yr(2015)

In [ ]:
nodos_base = get_nodos()
tooltips_energias = generar_tooltips_energias(data)
tooltips_centros = generar_tooltips_centros(data)
data_oferta_simple = eliminar_detalles_oferta(data)


In [ ]:
links2015 = make_links(2015)

In [ ]:
data_energias = data2015.reset_index().rename_axis({"index": "Nombre"}, 1)

In [ ]:
data_energias = nodos.merge(data_energias)

In [ ]:
CAMPOS_DATAJSON = {
    "ID": "id",
    "Nombre": "name",
    "ID Padre": "parent",
    "Grupo": "group",
    "Posicion": "position"
}

def generar_lista_nodos(nodos):
    return nodos.drop(
        ["Eje", "Es Base","Nombre Padre"], axis=1
    ).rename(columns=CAMPOS_DATAJSON
    ).to_dict(orient='records')


In [ ]:
lista_de_nodos = generar_lista_nodos(nodos)

In [ ]:
def get_id_from_nombre(nombre_nodo):
    return [n for n in lista_de_nodos if n["name"] == nombre_nodo][0].get("id")

In [ ]:
def convert_names_to_ids(links_list):
    return([{"source": get_id_from_nombre(link["source"]),
             "target": get_id_from_nombre(link["target"]),
             "value": round(link["value"],2)}
            for link in links_list if round(link["value"],2)>=0.01])

In [ ]:
lista_de_links_con_id = convert_names_to_ids(links2015)

In [ ]:
datajson_2015 = {"nodes": lista_de_nodos, "links": lista_de_links_con_id}

In [ ]:
def write_json(obj, path):
    """Escribo un objeto a un archivo JSON con codificación UTF-8."""
    obj_str = json.dumps(obj, indent=4, separators=(",", ": "),
                         ensure_ascii=False)
    with io.open(path, "w", encoding='utf-8') as target:
        target.write(obj_str)


In [ ]:
nodos = get_nodos("df")
lista_de_nodos = generar_lista_nodos(nodos)
for i in panel.items:
    data = get_yr(i)
    links = make_links(i)
    lista_de_links_con_id = convert_names_to_ids(links)
    datajson = {"nodes": lista_de_nodos, "links": lista_de_links_con_id}
    write_json(datajson, "output/data_{}.json".format(i))